# Exercício: Descoberta de Tópicos em E-mails Corporativos
* **Descrição:** Extraia 5 temas dominantes nos e-mails da Enron para obter visão geral dos assuntos tratados internamente.
* **Dataset:** Enron Email Dataset (500 k mensagens de email de uma empresa real, que foi publicada enquanto a empresa estava sendo investigada)
  * **Mais Informações:** https://www.cs.cmu.edu/~enron/
  * **Download:** https://www.kaggle.com/datasets/wcukierski/enron-email-dataset

## Passo a passo
1. Descarregar emails.csv (colunas message, subject, date, user).
2. Filtrar apenas “sent” e “inbox”; amostrar 20 000 linhas para caber em RAM de portátil.
3. Pré-processar: minúsculas, remover URLs, stop-words, lematizar (spacy-pt).
4. CountVectorizer → matriz BoW (max_features=20 000).
5. LatentDirichletAllocation(n_components=5, max_iter=10); ajustar modelo.
6. Mostrar top-10 palavras de cada tópico e atribuir rótulos de negócio.
7. Calcular distribuição de tópicos por utilizador e visualizar em gráfico de barras.

## Download do Dataset
* Para facilitar, fiz o upload do arquivo .csv no Google drive, link: https://drive.google.com/file/d/1FiIb0dliKf18NsbSCZmz27xXy0MXV8Iq/view
* Para fazer download é só executar o comando mais abaixo
* Caso o link ou o comando abaixo não funcionem, tem de fazer o download direto do próprio kaggle

In [ ]:
# Opcional: Se quizer fazer o download do arquivo direto do meu Google Drive

# NOTA: Em ambiente do Google colab o gdown já funciona diretamente
# caso necessário pode instalá-lo via pip, descomentanto a linha abaixo:
# !pip install gdown

!gdown 1FiIb0dliKf18NsbSCZmz27xXy0MXV8Iq -O emails.csv

In [ ]:
# Dependências
#!pip install pandas scikit-learn spacy==3.7.2 spacy-lookups-data pyLDAvis
!python -m spacy download pt_core_news_sm

In [ ]:
import pandas as pd, re, spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# 1. carregar
# NOTA: O dataset completo inclui mais de 500 mil emails
# Para uma análise inicial pode ser interessante carregar uma amostra menor
# por questões de tempo e performance
amostra = 20000

df = pd.read_csv('emails.csv')
df = df[df['file'].str.contains('sent|inbox', na=False)].sample(amostra, random_state=42)

## Pré-processamento
* Utilizaremos regex para tirar links do texto do email e conver ter para minúscula
* Utilizaremos um modelo de linguagem da spacy para fazer limpezas, tratamentos e anotacões no texto de cada email
* Chamar nlp(txt) processa o texto de entrada txt usando o pipeline do spaCy carregado. Tokeniza o texto, realiza anotações linguísticas (exceto NER e parsing, que foram desativados) e retorna um objeto Doc contendo esta informação processada. Este objeto Doc pode então ser usado para extrair as features desejadas, como lemas para modelagem de tópicos.

In [ ]:
# 2–3. pré-processamento
nlp = spacy.load('pt_core_news_sm', disable=['ner', 'parser'])
def limpa(txt):
    txt = re.sub(r'https?\S+', '', str(txt).lower())
    doc = nlp(txt)
    tokens = [t.lemma_ for t in doc if t.is_alpha and not t.is_stop]
    return ' '.join(tokens)
df['clean'] = df['message'].apply(limpa)

In [ ]:
# 4. vectorização
vec = CountVectorizer(max_features=20000)
X = vec.fit_transform(df['clean'])

In [ ]:
# 5. LDA
lda = LatentDirichletAllocation(n_components=5, max_iter=10, learning_method='online', random_state=0)
lda.fit(X)

In [ ]:
# 6. top palavras por tópico
palavras = vec.get_feature_names_out()
for k, comp in enumerate(lda.components_):
    print(f'\nTópico {k}:', ', '.join(palavras[i] for i in comp.argsort()[-10:][::-1]))

# Pontos Adicionais Sugeridos


1.   Refretir e entender sobre a limpeza dos dados. Visualizar os dados antes e depois da limpeza
2.   Como adaptarias esse código para um outro caso de uso real?
3. Como melhorar a limpeza dos dados para ter palavras melhores?
4. Temos um "gargalo" na parte de invocação do modelo do spacy para mexer no input...O que pode ser feito para melhorar?